# imports

In [ ]:
import pandas as pd

import sqlalchemy as sa
from sqlalchemy import func
from sqlalchemy.orm import Query

import src
from src.connect import create_wos_session, create_sqlite_session
from src.models import Author, ItemAuthorInstitution, Item, Source, Base, Abstract

# setup

In [ ]:
engine, s = create_wos_session()

In [ ]:
sources = [
    "clinical psychology review",
    "current directions in psychological science",
    "developmental review",
    "educational psychologist",
    "educational psychology review",
    "international review of sport and exercise psychology",
    "journal of abnormal psychology",
    "journal of applied psychology",
    "journal of consumer psychology",
    "journal of occupational health psychology",
    "journal of organizational behavior",
    "journal of personality and social psychology",
    "journal of the learning sciences",
    "leadership quarterly",
    "neuroscience and biobehavioral reviews",
    "personality and social psychology review",
    "personnel psychology",
    "perspectives on psychological science",
    "psychological bulletin",
    "psychological methods",
    "psychological review",
    "psychological science",
    "psychological science in the public interest",
    "social issues and policy review",
    "trends in cognitive sciences",
]

In [ ]:
len(sources)

25

In [ ]:
base_query = (
    s.query(Source)
    .join(Item)
    .join(ItemAuthorInstitution)
    .join(Author)
    .join(Abstract, isouter=True)
    .filter(
        Item.pubyear.between(2015, 2020),
        Item.doctype == "Article",
        func.lower(Source.sourcetitle).in_(sources),
    )
)

In [ ]:
base_query.with_entities(Author.pk_authors).distinct().count()

16546

In [ ]:
base_query.with_entities(Item.pk_items).distinct().count()

5629

In [ ]:
base_query.with_entities(Source.pk_sources).distinct().count()

24

In [ ]:
# check for non-existing sources
new = {
    c.lower() for (c,) in base_query.with_entities(Source.sourcetitle).distinct().all()
}

set(sources) - new

{'trends in cognitive sciences'}

In [ ]:
base_query.with_entities(ItemAuthorInstitution.pk_itm_auth_inst).distinct().count()

32385

In [ ]:
base_query.with_entities(Abstract.pk_abstracts).distinct().count()

32385

## Erstelle SQlite DB

In [ ]:
sqlite_path = src.PATH / "data/example.db"

if sqlite_path.is_file():
    sqlite_path.unlink()

In [ ]:
def copy_table(table):
    # nur zur Veranschaulichung (nicht notwendig):
    global engine
    global sqlite_engine

    df = pd.read_sql(base_query.with_entities(table).distinct().statement, engine)
    df.to_sql(
        getattr(table, "__tablename__"), sqlite_engine, if_exists="append", index=False
    )

In [ ]:
sqlite_engine, sqlite_s = create_sqlite_session(sqlite_path)

In [ ]:
Base.metadata.create_all(bind=sqlite_engine)

### Author

In [ ]:
df = pd.read_sql(base_query.with_entities(Author).distinct().statement, engine)
df.to_sql("authors", sqlite_engine, if_exists="append", index=False)

### Item

In [ ]:
df = pd.read_sql(base_query.with_entities(Item).distinct().statement, engine)
df.to_sql("items", sqlite_engine, if_exists="append", index=False)

### Source

In [ ]:
df = pd.read_sql(base_query.with_entities(Source).distinct().statement, engine)
df.to_sql("sources", sqlite_engine, if_exists="append", index=False)

### ItemAuthorInstitution

In [ ]:
df = pd.read_sql(
    base_query.with_entities(ItemAuthorInstitution).distinct().statement, engine
)
df.to_sql("items_authors_institutions", sqlite_engine, if_exists="append", index=False)

### Abstracts

In [ ]:
# man kann leider kein DISTINCT() auf CLOB Spalten anwenden, daher hier ein 'workaround'

unique_pk_abstracts = base_query.with_entities(Abstract.pk_abstracts).distinct()

query = s.query(Abstract).filter(Abstract.pk_abstracts.in_(unique_pk_abstracts))

df = pd.read_sql(query.statement, engine)
df.to_sql("abstracts", sqlite_engine, if_exists="append", index=False)

KeyboardInterrupt: 

# Ein Schmankerl für die Nerds

In [ ]:
def explain_query(query, detail="TYPICAL"):
    """show the query execution plan for an oracle DB

    possible values for detail (in ascending detail order):
    BASIC, SERIAL, TYPICAL, ALL
    """
    raw_query = str(
        query.statement.compile(
            engine,
            compile_kwargs={
                "literal_binds": True,
            },
        )
    )

    s.execute("ALTER SESSION SET current_schema = WOS_B_2020")
    s.execute("EXPLAIN PLAN FOR " + raw_query)
    out = s.execute(
        f"SELECT * FROM TABLE(DBMS_XPLAN.DISPLAY(format => '{detail}'))"
    ).fetchall()

    print("\n".join(str(row) for (row,) in out))

In [ ]:
explain_query(base_query, detail="SERIAL")

Plan hash value: 1451258128
 
------------------------------------------------------------------------------------------------------------------------------
| Id  | Operation                               | Name                       | Rows  | Bytes |TempSpc| Cost (%CPU)| Time     |
------------------------------------------------------------------------------------------------------------------------------
|   0 | SELECT STATEMENT                        |                            | 61079 |    10M|       | 94675   (1)| 00:00:10 |
|*  1 |  HASH JOIN                              |                            | 61079 |    10M|    11M| 94675   (1)| 00:00:10 |
|   2 |   NESTED LOOPS                          |                            | 61079 |    10M|       | 68759   (1)| 00:00:07 |
|*  3 |    HASH JOIN OUTER                      |                            |  9964 |  1595K|       | 28900   (1)| 00:00:03 |
|   4 |     NESTED LOOPS                        |                            |  9

In [ ]:
import pandas as pd
from sqlalchemy.orm import Query

from src.connect import create_wos_session
from src.models import Author, ItemAuthorInstitution, Item

In [ ]:
query = """
SELECT 
    a.PK_AUTHORS, a.FULLNAME, a.LASTNAME, a.FIRSTNAME,
    i.PK_ITEMS, i.ARTICLE_TITLE, i.PUBYEAR,
    s.SOURCETITLE
    FROM 
        wos_b_2020.SOURCES s
    LEFT JOIN
        wos_b_2020.ITEMS i
        ON
            s.PK_SOURCES = i.FK_SOURCES
    LEFT JOIN
        wos_b_2020.ITEMS_AUTHORS_INSTITUTIONS iai
        ON
            i.PK_ITEMS = iai.FK_ITEMS
    LEFT JOIN
        wos_b_2020.AUTHORS a
        ON
            iai.FK_AUTHORS = a.PK_AUTHORS
    WHERE 
        lower(s.SOURCETITLE) in (
            'astronomy and astrophysics review',
            'living reviews in solar physics',
            'astrophysical journal letters',
            'astrophysical journal, supplement series',
            'nature astronomy',
            'astronomical journal',
            'space science reviews',
            'astrophysical journal',
            'publications of the astronomical society of the pacific',
            'astronomy and astrophysics',
            'monthly notices of the royal astronomical society: letters',
            'monthly notices of the royal astronomical society',
            'publication of the astronomical society of japan',
            'new astronomy reviews',
            'icarus',
            'physics of the dark universe',
            'astrodynamics',
            'publications of the astronomical society of australia',
            'chinese physics c',
            'acta astronomica',
            'frontiers in astronomy and space sciences',
            'astroparticle physics',
            'revista mexicana de astronomia y astrofisica',
            'physics of the earth and planetary interiors',
            'journal of high energy astrophysics',
            'molecular astrophysics',
            'solar physics',
            'journal of cosmology and astroparticle physics',
            'international journal of modern physics d',
            'experimental astronomy'
            )
    AND
        i.PUBYEAR BETWEEN 2019 AND 2020
"""